In [ ]:
# workout_nse.py
# Program for normalizing returns from existing orders

from z_helper import *
util.startLoop()

# from json
a = assign_var('nse') + assign_var('common')
for v in a:
    exec(v)

with get_connected('nse', 'live') as ib:
    pos = ib.positions()
    trades = ib.trades()
    pos_contracts = ib.qualifyContracts(*[Contract(conId=c) for c in [p.contract.conId for p in pos]])
    pos_tickers = ib.reqTickers(*pos_contracts)
    pos_prices = {t.contract.conId: t.marketPrice() for t in pos_tickers}
    
#... position dataframe
pos_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'lastTradeDateOrContractMonth', 'strike', 'right']
pos_df = util.df(p.contract for p in pos)[pos_cols]
pos_df = pos_df.rename({'lastTradeDateOrContractMonth': 'expiry'}, axis='columns')
pos_df = pos_df.assign(dte=pos_df.expiry.apply(get_dte))
pos_df['position'] = [p.position for p in pos]
pos_df['avgCost'] = [p.avgCost for p in pos]
pos_df['close'] = pos_df.conId.map(pos_prices)

# take maximum of precision and minimum of hvstPrice, avgCost
pos_df = pos_df.assign(hvstPrice=np.maximum(np.minimum((pos_df.dte.apply(hvstPricePct)*pos_df.avgCost).apply(lambda x: get_prec(x, 0.05)), pos_df.close), prec))

#... trades dataframe
trades_df = util.df(t.contract for t in trades).join(util.df(t.order for t in trades)).join(util.df(t.orderStatus for t in trades), lsuffix='_')

# join with other parameters in sized.pkl
trades_df = trades_df.set_index('conId').join(pd.read_pickle(fspath+'sized.pkl')[['optId', 'close', 'undPrice', 'rom', 'stDev']].set_index('optId')).rename_axis('conId').reset_index()
trades_df = trades_df.rename({'lastTradeDateOrContractMonth': 'expiry'}, axis=1)
trades_df = trades_df.assign(sd=abs(trades_df.strike-trades_df.undPrice)/trades_df.stDev)

trades_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'expiry', 'strike', 'right', 'undPrice', 'sd', 'rom', 'orderId', 'permId', 'action', 'totalQuantity', 'close', 'lmtPrice', 'status']
trades_df = trades_df[trades_cols]

#... place BUY orders for positions without BUY orders

# get the pos conIds that have some trade (BUY or SELL)
pos_trade_cids = [tid for tid in trades_df.conId if tid in list(pos_df.conId)]

# remove pos conIds that have BUY trade action
pos_buy_cids = list(trades_df[trades_df.conId.isin(pos_trade_cids) & (trades_df.action == 'BUY') & (trades_df.status != 'Cancelled')].conId)

pos_buy_df = pos_df[~pos_df.conId.isin(pos_buy_cids)]

# prepare to place orders
buy_orders = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) 
                          for position, hvstPrice in zip(pos_buy_df.position, pos_buy_df.hvstPrice)]
buy_contracts = [{p.conId:p for p in pos_contracts}[pid] for pid in pos_buy_df.conId]

In [ ]:
#... see the best rom, based on latest price

# extract trades that are submitted
sub_df = trades_df[trades_df.status == 'Submitted'].drop('close', 1)

# fliter our the BUYs
sub_df = sub_df[sub_df.action == 'SELL']

# get the latest option prices
opt_list = [Option(i.symbol, i.expiry, i.strike, i.right, 'NSE') for i in sub_df[['symbol', 'expiry', 'strike', 'right']].itertuples()]

opt_contracts = []
with get_connected('nse', 'live') as ib:
    print("Qualifying option contracts ...")
    opt_contracts = ib.qualifyContracts(*opt_list)
    ticker = ib.reqTickers(*opt_contracts)

df_prices = pd.DataFrame({t.contract.conId: {'bid':t.bid, 'ask':t.ask, 'close':t.close} for t in ticker}).T

sub_df1 = sub_df.set_index('conId').join(df_prices)

sub_df1 = sub_df1.assign(lmtPrice1=(sub_df1.close+(sub_df1.ask-sub_df1.close)/2).apply(lambda x: get_prec(x, prec)))

In [ ]:
sub_df1.assign(delta=sub_df1.lmtPrice1/sub_df1.lmtPrice).sort_values('delta', ascending=False)